# Modulation and demodulation of an audio signal using the built-in soundcard

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
import IPython
from scipy.io import wavfile

In [ ]:
plt.rcParams["figure.figsize"] = (9,2.5)

The following functions can be used as a black box for now

In [ ]:
# Simple second order lowpass filter

def LPF(fc, sf, Q=(1/np.sqrt(2))):
    """Biquad lowpass filter"""
    w = 2 * np.pi * fc / sf
    alpha = np.sin(w) / (2 * Q)
    c = np.cos(w)
    a = np.array([1 + alpha, -2 * c, 1 - alpha])
    b = np.array([(1 - c) / 2, 1 - c, (1 - c) / 2])
    return b / a[0], a / a[0]

In [ ]:
def magnitude_response(b, a, sf, points=None, color='C1'):
    L = (points or max(len(a), len(b))) // 2
    points = 2 * L + 1
    w = 2 * np.pi * np.arange(-L, L+1) / points
    A, B = np.zeros(points, dtype='complex'), np.zeros(points, dtype='complex')
    for n, bn in enumerate(b):
        B += bn * np.exp(-1j * n * w)
    for n, an in enumerate(a):
        A += an * np.exp(-1j * n * w)
    A, B = np.abs(A), np.abs(B)
    M = B / np.where(A == 0, 1e-20, A)
    f = w / np.pi * sf / 2
    plt.plot(f, M, color, lw=2)

In [ ]:
def plot_mag_spec(data, Fs=2*np.pi):
    pts = len(data)
    w = Fs * (np.arange(0, pts) / pts - 0.5)
    X = np.abs(np.fft.fftshift(np.fft.fft(data, pts)))
    plt.plot(w, X);

In [ ]:
# default sampling frequency for notebook
fs = 96000

In [ ]:
# load audio file sampled at 96K. Actual audio is bandlimited to 8 kHz (max positive frequency)

audio_fs, audio = wavfile.read("audio96.wav")
assert audio_fs == fs, f'must use audio at {fs} Hz'
audio /= np.max(np.abs(audio))

plot_mag_spec(audio, Fs=fs)

In [ ]:
IPython.display.Audio(audio, rate=fs)

In [ ]:
# we modulate the audio signal using a carrier at 38 kHz. Positive bandwidth will be [30 kHz, 96 kHz[

carrier_freq = 38000

am_carrier = np.cos(2 * np.pi * carrier_freq / fs * np.arange(0, len(audio)))
am_signal = am_carrier * audio

plot_mag_spec(am_signal, Fs=fs)

In [ ]:
# modulated audio is outside of hearing range

IPython.display.Audio(am_signal, rate=fs)

In [ ]:
# demodulation: we can recover the baseband audio via multiplication by the carrier
#  we still have the cross-modulation sidebands that are disturbing since they overlap the hearing range

# note that, because of aliasing, the center frequency of the sidebands is (38 kHz * 2) wrapped over the [-48 kHz, 48 kHz] interval
# 38 * 2 = 76; 76 - 96 = -20. Sidebands are at +20 kHz and -20 kHz

bb_signal = am_carrier * am_signal
plot_mag_spec(bb_signal, Fs=fs)

In [ ]:
IPython.display.Audio(bb_signal, rate=fs)

In [ ]:
# let's use a simple second-order IIR lowpass to remove the sidebands:

b, a = LPF(3000, fs)
magnitude_response(b, a, fs, points=1000)
plot_mag_spec(bb_signal / 5000, Fs=fs)

In [ ]:
# final demodulation

demod = sp.lfilter(b, a, bb_signal)

plot_mag_spec(demod, Fs=fs)

In [ ]:
IPython.display.Audio(demod, rate=fs)